In [1]:
import os
from bs4 import BeautifulSoup
import nltk
#nltk.download('punkt')
from nltk.corpus import stopwords
from keybert import KeyBERT
import json
import tqdm
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer

In [2]:

# Ruta de la carpeta que contiene los archivos origen
folder_path = "C:/Users/usuario/Downloads/dataset_splitted/splits/train/student"  

# Lista creada para almacenar el texto del cuerpo de todas las páginas
all_page_texts = []

# Función para extraer el texto del cuerpo de la página
def extract_body_text(file_path):
    with open(file_path, 'r', encoding='latin-1') as file:
        text = file.read()

    # Se analiza el contenido HTML con BeautifulSoup
    soup = BeautifulSoup(text, 'html.parser')

    # Se extrae el texto del cuerpo de la página
    page_text = ' '.join(soup.stripped_strings)
    page_text = page_text.replace('\n', '')

    return page_text

# Se procesan los archivos y se guarda el texto del cuerpo en la lista all_page_texts
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path):  
        page_text = extract_body_text(file_path)
        all_page_texts.append(page_text)


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/usuario/Downloads/dataset_splitted/splits/train/student'

In [ ]:
#Se usa keybert para obtener las palabras clave de todos los textos.
modelo = KeyBERT()
palabras_clave = modelo.extract_keywords(all_page_texts, stop_words='english')

#Ver las palabras clave encntradas
for keyword in palabras_clave:
    print(keyword)
    
todas_palabras_clave = [clave for lista_p_claves in palabras_clave for clave in lista_p_claves]

In [ ]:
# Se usa un diccionario para poder calcular la suma del valor de cada palabra clave cuando se encuentra varias veces.
sums = {}

# Se itera sobre las palabras clave encontradas y se suman sus valores
for clave in todas_palabras_clave:
    palabra, valor = clave  
    if palabra in sums:
        sums[palabra] += valor
    else:
        sums[palabra] = valor

# Se ordenan las palabras clave encontradas por el resultado de sus sumas de manera descendente
suma_ordenada = sorted(sums.items(), key=lambda x: x[1], reverse=True)

# Imprimir las palabras clave con sus sumas
for palabra, suma in suma_ordenada:
    print(f"Palabra clave: {palabra}, Suma: {suma}")


In [ ]:
# Se crea una lista para todas las palabras clave
palabras_clave_lista = [palabra for palabra, suma in suma_ordenada]

# Ruta de la carpeta de salida para el archivo JSON
output_folder = "C:/Users/usuario/Downloads/dataset_splitted/splits/train_json/student_json"
os.makedirs(output_folder, exist_ok=True)

# Ruta completa al archivo JSON en la carpeta de salida
output_json_file = os.path.join(output_folder, "keywords.json")

# Se crea un diccionario para almacenar las palabras y el ground_truth
data_json = {"words": palabras_clave_lista, "ground_truth": "student"}

# Se guarda el diccionario en un archivo JSON 
with open(output_json_file, 'w', encoding='utf-8') as json_file:
    json.dump(data_json, json_file, ensure_ascii=False, separators=(',', ':'))

print(f"La lista de palabras clave se ha guardado en '{output_json_file}'")



In [ ]:
!pip install gensim

In [3]:
import json
import csv
import os

def get_file_2(path_file):  
  #print(path_file)
  with open(path_file) as f:
      data = json.load(f)
  #print(data.keys())
  dict_text={key: data[key] for key in data.keys() if key in {'title','summary','links'} }
  try:
    dict_text['keyword']=data['keyword_frequency_kebert']
  except:
    print(data.keys())
  return dict_text,data['ground_truth']

def get_class(train_path):
  #print(train_path)
  if os.path.exists(train_path) and os.path.isdir(train_path):
    carpetas = [nombre for nombre in os.listdir(train_path) if os.path.isdir(os.path.join(train_path, nombre))]
    return carpetas
  else:
    print("El directorio especificado no existe o no es un directorio válido.")

def get_archives(train_path):
  if os.path.exists(train_path) and os.path.isdir(train_path):
    carpetas = [os.path.join(train_path,nombre) for nombre in os.listdir(train_path) if os.path.isfile(os.path.join(train_path, nombre))]
    return carpetas
  else:
    print("El directorio especificado no existe o no es un directorio válido.")

def get_all_files(path_data,max_cont=-1):

    #'./data/splits/train_json'
    classes=get_class(path_data)
    archiver_per_clases={}
    prompt_per_clases={}
    cantidad_archivos={}
    promt_all_reson=[]
    archiver_all_reson=[]
    if len(classes)>0:
      for class_name in classes:
        #print(get_archives(os.path.join(path_data,i)))
        archiver_per_clases[class_name]=get_archives(os.path.join(path_data,class_name))[0:max_cont]
        prompt_per_clases[class_name]=[get_file_2(archive) for archive in  archiver_per_clases[class_name]]
        promt_all_reson.extend(prompt_per_clases[class_name])
        archiver_all_reson.extend(archiver_per_clases[class_name])

        #print(len(archiver_per_clases[class_name]),promt_all_reson)
        cantidad_archivos[class_name]=len(archiver_per_clases[class_name])
    else:
       archiver_per_clases['test']=get_archives(os.path.join(path_data,class_name))
       prompt_per_clases['test']=[get_file_2(archive) for archive  in  archiver_per_clases['test']]

    # Clave especial para ir de último
    clave_especial = 'other'
    # Ordenar el diccionario por el valor, manteniendo 'other' al final si existe
    ordenado = sorted(cantidad_archivos.items(), key=lambda item: (item[0] == clave_especial, -item[1]))

    ordenado =[i[0] for i in ordenado]
    return archiver_per_clases,prompt_per_clases,promt_all_reson,archiver_all_reson,ordenado
    

In [4]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from gensim import corpora, models
import gensim
import string

# Descargar stopwords de NLTK
nltk.download('stopwords')
nltk.download('wordnet')

# Tus textos
textos = [
    "Este es el primer texto de ejemplo.",
    "Aquí tienes otro ejemplo de texto.",
    "Este será el tercer texto de muestra.",
    # Agrega más textos según sea necesario
]

# Preprocesamiento
stop = set(stopwords.words('english'))  # Ajusta el idioma de acuerdo a tus textos
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

def limpiar_documento(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalizado = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalizado

doc_limpio = [limpiar_documento(doc).split() for doc in textos]

# Crear diccionario y matriz de términos del documento
diccionario = corpora.Dictionary(doc_limpio)
doc_term_matrix = [diccionario.doc2bow(doc) for doc in doc_limpio]

# Crear el modelo LDA
ldamodel = gensim.models.ldamodel.LdaModel(doc_term_matrix, num_topics=3, id2word = diccionario, passes=50)

# Mostrar los tópicos
print(ldamodel.print_topics(num_topics=3, num_words=3))

# Identificar el texto que mejor representa el tópico más común
topicos = ldamodel.get_document_topics(doc_term_matrix)
topicos_principales = [max(tp, key=lambda item: item[1])[0] for tp in topicos]

# Asignar cada texto a su tópico principal
textos_topicos = zip(enumerate(textos), topicos_principales)

# Encontrar el texto más representativo para el tópico más común
#topico_mas_comun = max(set(topicos_principales), key = topicos_principales.count)
#textos_representativos = [texto for texto, topico in textos_topicos if topico == topico_mas_comun ]


umbral_relevancia = 0.5  # Solo considerar textos donde el tópico común represente al menos el 50% 




[nltk_data] Downloading package stopwords to
[nltk_data]     /home/alourido/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/alourido/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


[(0, '0.077*"texto" + 0.077*"de" + 0.077*"el"'), (1, '0.135*"ejemplo" + 0.135*"texto" + 0.135*"de"'), (2, '0.118*"el" + 0.118*"este" + 0.117*"texto"')]


In [5]:
train,promt_train,b_2,d_2,_=get_all_files('../data/splits/train_new_data',-1)

In [6]:
print(_)

['student', 'faculty', 'course', 'project', 'department', 'staff', 'other']


In [7]:
print(train.keys())

dict_keys(['staff', 'project', 'other', 'faculty', 'department', 'student', 'course'])


In [8]:
def tranform_text(text):
  return ' '.join([str(text[key]) for key in text.keys()])

In [9]:
def get_most_representative_text(promt):
    # Tus textos
    textos =promt
    
    # Preprocesamiento
    stop = set(stopwords.words('english'))  # Ajusta el idioma de acuerdo a tus textos
    exclude = set(string.punctuation)
    lemma = WordNetLemmatizer()
    
    def limpiar_documento(doc):
        stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
        punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
        normalizado = " ".join(lemma.lemmatize(word) for word in punc_free.split())
        return normalizado
    
    doc_limpio = [limpiar_documento(doc).split() for doc in textos]
    
    # Crear diccionario y matriz de términos del documento
    diccionario = corpora.Dictionary(doc_limpio)
    doc_term_matrix = [diccionario.doc2bow(doc) for doc in doc_limpio]
    
    # Crear el modelo LDA
    ldamodel = gensim.models.ldamodel.LdaModel(doc_term_matrix, num_topics=3, id2word = diccionario, passes=50)
    
    # Mostrar los tópicos
    print(ldamodel.print_topics(num_topics=20, num_words=20))
    
    # Identificar el texto que mejor representa el tópico más común
    topicos = ldamodel.get_document_topics(doc_term_matrix)
    topicos_principales = [max(tp, key=lambda item: item[1])[0] for tp in topicos]
    
    # Asignar cada texto a su tópico principal
    textos_topicos = zip(textos, topicos_principales)
    
    # Encontrar el texto más representativo para el tópico más común
    topico_mas_comun = max(set(topicos_principales), key = topicos_principales.count)
    #textos_representativos = [texto for texto, topico in textos_topicos if topico == topico_mas_comun]
        
    umbral_relevancia = 0.5  # Solo considerar textos donde el tópico común represente al menos el 50% 
    
    # Crear tuplas con el índice, el texto, el tópico principal y la proporción del tópico principal
    textos_topicos_indices = [(indice, texto, topico, prop) for indice, (texto, tp) in enumerate(zip(textos, topicos)) for topico, prop in [max(tp, key=lambda item: item[1])]]
    
    # Ahora el desempaquetado debería funcionar correctamente
    textos_representativos_indices = [(indice,texto,prop) for indice, texto, topico, prop in textos_topicos_indices if topico == topico_mas_comun and prop >= umbral_relevancia]

    return sorted(textos_representativos_indices, key=lambda item: item[2], reverse=True)

In [10]:
promt=[tranform_text(i[0]) for i in promt_train['student']]
student_text=get_most_representative_text(promt)
print(student_text,len(student_text))

[(0, '0.015*"1996" + 0.014*"page" + 0.014*"gmt" + 0.012*"home" + 0.010*"computer" + 0.009*"university" + 0.009*"science" + 0.009*"research" + 0.008*"server" + 0.007*"nov" + 0.007*"contenttype" + 0.007*"texthtml" + 0.007*"date" + 0.007*"contentlength" + 0.007*"lastmodified" + 0.006*"computing" + 0.006*"department" + 0.005*"system" + 0.005*"tue" + 0.005*"information"'), (1, '0.028*"page" + 0.025*"gmt" + 0.024*"home" + 0.015*"server" + 0.014*"1996" + 0.013*"date" + 0.013*"texthtml" + 0.013*"contenttype" + 0.013*"computer" + 0.012*"lastmodified" + 0.012*"contentlength" + 0.011*"science" + 0.011*"university" + 0.009*"nov" + 0.007*"department" + 0.007*"10" + 0.006*"mimeversion" + 0.006*"student" + 0.005*"cern30" + 0.005*"thu"'), (2, '0.023*"page" + 0.018*"home" + 0.016*"gmt" + 0.013*"1996" + 0.012*"computer" + 0.011*"university" + 0.010*"science" + 0.009*"server" + 0.009*"date" + 0.008*"contenttype" + 0.008*"texthtml" + 0.008*"nov" + 0.007*"lastmodified" + 0.007*"contentlength" + 0.007*"home

In [11]:
#print(train['staff'][student_text[0][0]])
print(train['student'][student_text[0][0]],student_text[0][0])
print(train['student'][student_text[0][0]],student_text[0][0],len(str(promt_train['student'][student_text[0][0]])))
print(train['student'][student_text[1][0]],student_text[1][0],len(str(promt_train['student'][student_text[1][0]])))
print(train['student'][student_text[2][0]],student_text[2][0],len(str(promt_train['student'][student_text[2][0]])))

../data/splits/train_new_data/student/xridnc.json 1036
../data/splits/train_new_data/student/xridnc.json 1036 1358
../data/splits/train_new_data/student/dadrnar.json 293 1294
../data/splits/train_new_data/student/ukdagdfa.json 1265 1253


In [12]:
promt=[tranform_text(i[0]) for i in promt_train['department']]
student_department=get_most_representative_text(promt)
print(student_department,len(student_department))

[(0, '0.030*"science" + 0.030*"computer" + 0.026*"department" + 0.015*"page" + 0.014*"gmt" + 0.014*"c" + 0.014*"1996" + 0.013*"university" + 0.012*"server" + 0.011*"home" + 0.010*"nov" + 0.009*"date" + 0.009*"contenttype" + 0.009*"texthtml" + 0.008*"engineering" + 0.008*"information" + 0.007*"course" + 0.007*"tue" + 0.007*"program" + 0.007*"26"'), (1, '0.043*"science" + 0.039*"computer" + 0.030*"department" + 0.023*"university" + 0.017*"server" + 0.017*"information" + 0.014*"gmt" + 0.013*"1996" + 0.013*"page" + 0.012*"course" + 0.012*"research" + 0.011*"student" + 0.011*"faculty" + 0.010*"c" + 0.009*"computing" + 0.009*"home" + 0.008*"graduate" + 0.008*"program" + 0.008*"nov" + 0.008*"web"'), (2, '0.007*"notre" + 0.007*"dame" + 0.006*"undergraduate" + 0.006*"byu" + 0.006*"course" + 0.005*"graduate" + 0.005*"dartmouth" + 0.005*"rensselaer" + 0.005*"eec" + 0.004*"listing" + 0.004*"cse" + 0.004*"csci" + 0.003*"engineering" + 0.003*"evaluation" + 0.003*"ucsb" + 0.003*"class" + 0.003*"multi

In [13]:
#print(train['staff'][student_text[0][0]])
print(train['department'][student_department[0][0]],student_department[0][0])

../data/splits/train_new_data/department/aijjcyf.json 143


In [14]:
promt=[tranform_text(i[0]) for i in promt_train['faculty']]
student_faculty=get_most_representative_text(promt)
print(student_faculty,len(student_faculty))

[(0, '0.013*"system" + 0.012*"computer" + 0.008*"science" + 0.008*"research" + 0.007*"page" + 0.006*"home" + 0.006*"software" + 0.006*"computing" + 0.005*"engineering" + 0.005*"program" + 0.004*"multimedia" + 0.004*"programming" + 0.004*"1996" + 0.004*"algorithm" + 0.004*"c" + 0.004*"university" + 0.004*"conference" + 0.003*"language" + 0.003*"database" + 0.003*"information"'), (1, '0.018*"gmt" + 0.017*"computer" + 0.016*"science" + 0.013*"university" + 0.012*"1996" + 0.011*"professor" + 0.011*"server" + 0.009*"date" + 0.009*"contenttype" + 0.009*"texthtml" + 0.009*"page" + 0.009*"research" + 0.008*"home" + 0.008*"lastmodified" + 0.008*"contentlength" + 0.008*"nov" + 0.007*"computing" + 0.007*"department" + 0.006*"c" + 0.005*"programming"'), (2, '0.019*"gmt" + 0.016*"computer" + 0.016*"1996" + 0.013*"science" + 0.012*"page" + 0.011*"server" + 0.011*"home" + 0.011*"university" + 0.010*"date" + 0.010*"contenttype" + 0.010*"texthtml" + 0.009*"nov" + 0.009*"research" + 0.009*"lastmodified"

In [15]:
#print(train['staff'][student_text[0][0]])
print(train['faculty'][student_faculty[0][0]],student_faculty[0][0])
print(train['faculty'][student_faculty[0][0]],student_faculty[0][0],len(str(promt_train['faculty'][student_faculty[0][0]])))
print(train['faculty'][student_faculty[1][0]],student_faculty[1][0],len(str(promt_train['faculty'][student_faculty[1][0]])))
print(train['faculty'][student_faculty[2][0]],student_faculty[2][0],len(str(promt_train['faculty'][student_faculty[2][0]])))

../data/splits/train_new_data/faculty/nfbkiigv.json 199
../data/splits/train_new_data/faculty/nfbkiigv.json 199 1804
../data/splits/train_new_data/faculty/linruc.json 212 1263
../data/splits/train_new_data/faculty/qcxwavy.json 514 1432


In [16]:
promt=[tranform_text(i[0]) for i in promt_train['course']]
student_course=get_most_representative_text(promt)
print(student_course,len(student_course))

[(0, '0.013*"1996" + 0.012*"gmt" + 0.010*"c" + 0.009*"computer" + 0.009*"homework" + 0.008*"date" + 0.008*"office" + 0.007*"nov" + 0.007*"server" + 0.007*"page" + 0.007*"texthtml" + 0.007*"contenttype" + 0.007*"programming" + 0.006*"course" + 0.006*"system" + 0.006*"due" + 0.006*"assignment" + 0.006*"syllabus" + 0.006*"lastmodified" + 0.005*"contentlength"'), (1, '0.016*"computer" + 0.015*"1996" + 0.013*"course" + 0.013*"gmt" + 0.012*"c" + 0.009*"page" + 0.009*"science" + 0.008*"server" + 0.008*"date" + 0.008*"lecture" + 0.007*"contenttype" + 0.007*"texthtml" + 0.007*"programming" + 0.007*"home" + 0.006*"nov" + 0.005*"lastmodified" + 0.005*"system" + 0.005*"contentlength" + 0.005*"introduction" + 0.005*"information"'), (2, '0.017*"assignment" + 0.016*"1996" + 0.014*"page" + 0.014*"programming" + 0.014*"gmt" + 0.013*"c" + 0.009*"home" + 0.009*"class" + 0.009*"course" + 0.008*"server" + 0.008*"nov" + 0.008*"date" + 0.008*"texthtml" + 0.008*"contenttype" + 0.007*"office" + 0.007*"syllabus

In [17]:
#print(train['staff'][student_text[0][0]])
print(train['course'][student_course[0][0]],student_course[0][0],len(str(promt_train['course'][student_course[0][0]])))
print(train['course'][student_course[1][0]],student_course[1][0],len(str(promt_train['course'][student_course[1][0]])))
print(train['course'][student_course[2][0]],student_course[2][0],len(str(promt_train['course'][student_course[2][0]])))
print(train['course'][student_course[3][0]],student_course[3][0],len(str(promt_train['course'][student_course[3][0]])))

../data/splits/train_new_data/course/nbiquru.json 254 3209
../data/splits/train_new_data/course/lczbz.json 191 2706
../data/splits/train_new_data/course/bvfjoth.json 517 2086
../data/splits/train_new_data/course/iaxei.json 444 2086


In [18]:
len(str(promt_train['course'][student_course[0][0]]))

3209

In [19]:
promt=[tranform_text(i[0]) for i in promt_train['project']]

student_project=get_most_representative_text(promt)
print(student_project,len(student_project))

[(0, '0.015*"gmt" + 0.012*"research" + 0.010*"server" + 0.010*"1996" + 0.010*"laboratory" + 0.009*"project" + 0.009*"group" + 0.009*"computer" + 0.008*"system" + 0.008*"robotics" + 0.008*"texthtml" + 0.008*"contenttype" + 0.008*"date" + 0.007*"university" + 0.007*"lastmodified" + 0.007*"contentlength" + 0.006*"nov" + 0.005*"page" + 0.005*"lab" + 0.004*"information"'), (1, '0.013*"system" + 0.013*"group" + 0.009*"gmt" + 0.009*"software" + 0.009*"research" + 0.008*"project" + 0.007*"server" + 0.007*"page" + 0.006*"computer" + 0.006*"parallel" + 0.006*"engineering" + 0.006*"home" + 0.005*"1996" + 0.005*"distributed" + 0.005*"date" + 0.005*"contenttype" + 0.005*"texthtml" + 0.005*"science" + 0.004*"performance" + 0.004*"lastmodified"'), (2, '0.016*"gmt" + 0.013*"project" + 0.013*"1996" + 0.012*"research" + 0.011*"page" + 0.010*"server" + 0.010*"system" + 0.009*"nov" + 0.008*"date" + 0.008*"contenttype" + 0.008*"texthtml" + 0.008*"home" + 0.008*"computer" + 0.008*"group" + 0.007*"computing"

In [20]:
#print(train['staff'][student_text[0][0]])
print(train['project'][student_project[0][0]],student_project[0][0])

../data/splits/train_new_data/project/tjjifi.json 320


In [21]:
promt=[tranform_text(i[0]) for i in promt_train['staff']]

student_staff=get_most_representative_text(promt)
print(student_staff,len(student_staff))

[(0, '0.016*"page" + 0.016*"gmt" + 0.012*"home" + 0.012*"1996" + 0.010*"computer" + 0.010*"server" + 0.010*"science" + 0.009*"university" + 0.008*"contenttype" + 0.008*"date" + 0.008*"texthtml" + 0.008*"nov" + 0.008*"research" + 0.008*"web" + 0.007*"contentlength" + 0.007*"lastmodified" + 0.006*"tue" + 0.005*"department" + 0.005*"26" + 0.004*"system"'), (1, '0.019*"gmt" + 0.016*"computer" + 0.016*"page" + 0.015*"1996" + 0.014*"server" + 0.013*"home" + 0.012*"science" + 0.010*"university" + 0.010*"department" + 0.010*"contenttype" + 0.010*"texthtml" + 0.010*"date" + 0.009*"lastmodified" + 0.009*"contentlength" + 0.009*"nov" + 0.008*"research" + 0.007*"tue" + 0.006*"26" + 0.005*"office" + 0.004*"m"'), (2, '0.011*"system" + 0.009*"page" + 0.008*"computer" + 0.008*"gmt" + 0.008*"home" + 0.008*"science" + 0.006*"1996" + 0.006*"server" + 0.005*"network" + 0.005*"research" + 0.005*"database" + 0.005*"department" + 0.004*"nov" + 0.004*"contentlength" + 0.004*"lastmodified" + 0.004*"texthtml" +

In [22]:
#print(train['staff'][student_text[0][0]])
print(train['staff'][student_staff[0][0]],student_staff[0][0])

../data/splits/train_new_data/staff/ullwnir.json 76


In [23]:
promt=[tranform_text(i[0]) for i in promt_train['other']]
student_other=get_most_representative_text(promt)
print(student_other,len(student_other))


[(0, '0.091*"000" + 0.043*"001" + 0.019*"1" + 0.013*"002" + 0.012*"13" + 0.010*"60457" + 0.008*"2" + 0.007*"003" + 0.004*"3" + 0.004*"authannot" + 0.004*"4" + 0.004*"6" + 0.004*"004" + 0.004*"5" + 0.003*"7" + 0.003*"005" + 0.003*"12" + 0.003*"14" + 0.003*"11" + 0.002*"17"'), (1, '0.008*"program" + 0.008*"gmt" + 0.008*"c" + 0.007*"1996" + 0.007*"assignment" + 0.006*"programming" + 0.006*"slide" + 0.005*"date" + 0.005*"system" + 0.005*"1" + 0.005*"server" + 0.005*"contenttype" + 0.005*"2" + 0.005*"texthtml" + 0.005*"lecture" + 0.004*"cse" + 0.004*"computer" + 0.004*"10" + 0.004*"project" + 0.004*"file"'), (2, '0.019*"gmt" + 0.013*"server" + 0.011*"1996" + 0.011*"page" + 0.011*"date" + 0.010*"contenttype" + 0.010*"texthtml" + 0.009*"contentlength" + 0.009*"lastmodified" + 0.008*"university" + 0.007*"computer" + 0.006*"science" + 0.006*"home" + 0.006*"tue" + 0.006*"10" + 0.005*"research" + 0.005*"information" + 0.004*"department" + 0.004*"web" + 0.004*"system"')]
[(468, 'EOSDIS Bookmarks o

In [24]:
print(train['other'][student_other[0][0]],student_other[0][0],len(str(promt_train['other'][student_other[0][0]])))
print(train['other'][student_other[1][0]],student_other[1][0],len(str(promt_train['other'][student_other[1][0]])))
print(train['other'][student_other[2][0]],student_other[2][0],len(str(promt_train['other'][student_other[2][0]])))
print(train['other'][student_other[3][0]],student_other[3][0],len(str(promt_train['other'][student_other[3][0]])))

../data/splits/train_new_data/other/mqwloql.json 468 7469
../data/splits/train_new_data/other/ucajmga.json 1086 6259
../data/splits/train_new_data/other/cnkdwoo.json 1006 5156
../data/splits/train_new_data/other/akhlinns.json 703 8499
